## **Database Connection**

In [57]:
import mysql.connector
from datetime import datetime
from tabulate import tabulate

db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="haphuong300505",
    database="CinemaDB"
)

cursor = db.cursor()

## **Booking System**

### **Book a seat with optional snack and discount**

In [58]:
import json

def book_tickets_with_optional_snack(customer_name, phone, email, payment_method,
                                     seat_list, snack_list=None, discount_code=None):
    try:
        seat_json = json.dumps(seat_list)
        snack_json = json.dumps(snack_list) if snack_list else None

        args = (
            customer_name,
            phone,
            email,
            payment_method,
            seat_json,
            snack_json,
            discount_code
        )

        cursor.callproc("BookTicketsWithOptionalSnack", args)
        db.commit()
        print("Booking successful!")

    except mysql.connector.Error as err:
        print("Error during booking:", err)


In [59]:
seat_list = [[9, 76], [9, 77]]
snack_list = [["Hotdog", 2], ["Soda", 1]]

book_tickets_with_optional_snack(
    customer_name="Ethan Kim",
    phone="5678901234",
    email="ethan@gmail.com",
    payment_method="Credit Card",
    seat_list=seat_list,
    snack_list=snack_list,
    discount_code="STUDENT15"
)


Booking successful!


### **Cancel booking by BookingID**

In [33]:
def cancel_booking(booking_id):
    try:
        cursor.callproc("CancelBooking", (booking_id,))
        db.commit()
        print("Booking cancelled successfully.")
    except mysql.connector.Error as err:
        print("Error:", err)

In [34]:
cancel_booking(11)

Booking cancelled successfully.


### **Apply discount code to existing BookingTicket**

In [35]:
def apply_discount_code(discount_code, booking_ticket_id):
    try:
        cursor.callproc("ApplyDiscountCode", (discount_code, booking_ticket_id))
        for result in cursor.stored_results():
            print(result.fetchall())
        db.commit()
    except mysql.connector.Error as err:
        print("Error:", err)

In [36]:
apply_discount_code("FLASH25", 1)

[('Giá sau khi giảm: 135,000.00',)]


### **Generate screening report**

In [37]:
from tabulate import tabulate

def get_screening_report():
    query = """
    SELECT 
        s.ScreeningID,
        m.MovieTitle,
        m.Genre,
        m.DurationMinutes,
        r.RoomName,
        r.RoomType,
        r.Capacity,
        s.ScreeningDate,
        s.ScreeningTime,
        COUNT(bt.BookingTicketsID) AS TicketsSold,
        COALESCE(SUM(tp.Price), 0) AS TotalTicketRevenue
    FROM Screenings s
    JOIN Movies m ON s.MovieID = m.MovieID
    JOIN CinemaRooms r ON s.RoomID = r.RoomID
    LEFT JOIN BookingTickets bt ON s.ScreeningID = bt.ScreeningID
    LEFT JOIN TicketPrice tp ON bt.PricingID = tp.PricingID
    GROUP BY s.ScreeningID, m.MovieTitle, m.Genre, m.DurationMinutes,
             r.RoomName, r.RoomType, r.Capacity, s.ScreeningDate, s.ScreeningTime
    ORDER BY s.ScreeningDate, s.ScreeningTime;
    """
    cursor.execute(query)
    rows = cursor.fetchall()
    headers = [
        "Screening ID", "Movie Title", "Genre", "Duration (min)",
        "Room", "Room Type", "Capacity", "Date", "Time", 
        "Tickets Sold", "Total Revenue"
    ]
    print(tabulate(rows, headers=headers, tablefmt="grid"))


In [38]:
get_screening_report()

+----------------+-----------------------------------------------------------------------+----------+------------------+--------+-------------+------------+------------+----------+----------------+-----------------+
|   Screening ID | Movie Title                                                           | Genre    |   Duration (min) | Room   | Room Type   |   Capacity | Date       | Time     |   Tickets Sold |   Total Revenue |
+================+=======================================================================+==========+==================+========+=============+============+============+==========+================+=================+
|              6 | A Minecraft Movie                                                     | Family   |              101 | Room F | VIP         |         10 | 2025-05-10 | 11:00:00 |              3 |          540000 |
+----------------+-----------------------------------------------------------------------+----------+------------------+--------+-------

### **Generate sales report**

In [45]:
def get_sales_report():
    query = """
    SELECT 
        DATE(p.PaymentDate) AS PaymentDate,
        COUNT(p.PaymentID) AS TotalPayments,
        COALESCE(SUM(tp.Price), 0) AS TicketRevenue,
        COALESCE(SUM(bs.TotalAmount), 0) AS SnackRevenue,
        SUM(p.Amount) AS TotalRevenue
    FROM Payments p
    LEFT JOIN Bookings b ON p.BookingID = b.BookingID
    LEFT JOIN BookingTickets bt ON b.BookingID = bt.BookingID
    LEFT JOIN TicketPrice tp ON bt.PricingID = tp.PricingID
    LEFT JOIN BookingSnacks bs ON b.BookingID = bs.BookingID
    WHERE p.Status = 'Paid'
    GROUP BY DATE(p.PaymentDate)
    ORDER BY PaymentDate DESC;
    """
    cursor.execute(query)
    raw_rows = cursor.fetchall()

    # Format rows to avoid scientific notation
    formatted_rows = [
        [
            row[0],                                
            row[1],                               
            f"{row[2]:,.0f}",              
            f"{row[3]:,.0f}",                     
            f"{row[4]:,.0f}"                   
        ]
        for row in raw_rows
    ]

    headers = [
        "Payment Date",
        "Total Successful Payments",
        "Ticket Revenue (VND)",
        "Snack Revenue (VND)",
        "Total Revenue (VND)"
    ]

    print(tabulate(formatted_rows, headers=headers, tablefmt="grid"))


In [46]:
get_sales_report()

+----------------+-----------------------------+------------------------+-----------------------+-----------------------+
| Payment Date   |   Total Successful Payments | Ticket Revenue (VND)   | Snack Revenue (VND)   | Total Revenue (VND)   |
+================+=============================+========================+=======================+=======================+
| 2025-05-17     |                           2 | 100,000                | 60,000                | 230,000               |
+----------------+-----------------------------+------------------------+-----------------------+-----------------------+
| 2025-05-14     |                           1 | 0                      | 0                     | 100,000               |
+----------------+-----------------------------+------------------------+-----------------------+-----------------------+
| 2025-05-13     |                           2 | 360,000                | 60,000                | 780,000               |
+----------------+------

In [47]:
cursor.close()
db.close()